In [1]:
import torch
import numpy as np
from tqdm import tqdm
from utils import Seq2SeqDataset, translate

# 1. Datasets

In [2]:
# Source seq
with open('../datasets/test/en', 'r+') as file_obj:
    test_en = file_obj.readlines()
test_en = [ line.strip().lower() for line in test_en ]

print(len(test_en))
print(test_en[:5])

101636
['ive been thinking about buying new towels.', 'ive been thinking about buying new towels.', 'base jumping is an extreme sport.', 'dancing is basically a rigorous sport.', "i don't know and i don't care."]


In [3]:
# Target seq
with open('../datasets/test/vi', 'r+') as file_obj:
    test_vi = file_obj.readlines()
test_vi = [ line.strip().lower() for line in test_vi ]

print(len(test_vi))
print(test_vi[:5])

101636
['tôi đang nghĩ về việc mua khăn mới.', 'tôi đã suy nghĩ về việc mua khăn mới.', 'nhảy cầu là một môn thể thao mạo hiểm.', 'nhảy cơ bản là một môn thể thao khắc nghiệt.', 'tôi không biết và tôi không quan tâm.']


# 2. Translate single sentence

In [4]:
device = torch.device(
    "cuda" if torch.cuda.is_available() else "cpu")

## Load model
transformer = torch.load('ckpts/8B_Transformer_en2vi.model.pth')
transformer = transformer.to(device)
transformer.eval()

## Load tokenizers
from transformers import AutoTokenizer

en_tokenizer = AutoTokenizer.from_pretrained(
    'bert-base-uncased')
vi_tokenizer = AutoTokenizer.from_pretrained(
    'vinai/phobert-base')

## Load test dset
Tx = 55
Ty = 60
test_dset = Seq2SeqDataset(
    en_tokenizer=en_tokenizer, vi_tokenizer=vi_tokenizer,
    Tx=Tx, Ty=Ty,
    en_sentences="../datasets/test/en",
    vi_sentences="../datasets/test/vi",
    mode="test", device=device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
test_size = len(test_dset)
for i in range(2):
    sample_idx = np.random.randint(0, test_size)

    x_utt = test_dset[sample_idx]['X_sentence']
    y_utt = test_dset[sample_idx]['Y_sentence']
    y_decodes = translate(x_utt,
        model=transformer,
        en_tokenizer=en_tokenizer, vi_tokenizer=vi_tokenizer,
        Tx=Tx, Ty=Ty, beam_width=10, device=device)
    
    print(f'[En]: {x_utt}')
    print(f'[Vi-Groundtruth]: {y_utt}')
    for x, (decode, att) in enumerate(y_decodes):
        print(f'\t[Vi-Translate {x}]: {decode}')

[En]: life in the city has never agreed with me.
[Vi-Groundtruth]: cuộc sống ở thành phố chưa bao giờ đồng ý với tôi.
	[Vi-Translate 0]: <s> cuộc sống ở thành phố chưa bao giờ nói chuyện với tôi. </s>
	[Vi-Translate 1]: <s> cuộc sống ở thành phố chưa bao giờ đồng ý với tôi. </s>
	[Vi-Translate 2]: <s> cuộc sống ở thành phố nhưng chưa bao giờ nói chuyện với tôi. </s>
	[Vi-Translate 3]: <s> cuộc sống ở thành phố chưa bao giờ nói với tôi. </s>
	[Vi-Translate 4]: <s> cuộc sống ở thành phố chưa bao giờ đồng ý. </s>
	[Vi-Translate 5]: <s> cuộc sống ở thành phố chưa bao giờ quyết định. </s>
	[Vi-Translate 6]: <s> cuộc sống ở thành phố chưa bao giờ nói chuyện. </s>
	[Vi-Translate 7]: <s> cuộc sống ở thành phố chưa bao giờ quyết định với tôi. </s>
	[Vi-Translate 8]: <s> cuộc sống ở thành phố nhưng chưa bao giờ nói chuyện. </s>
	[Vi-Translate 9]: <s> cuộc sống ở thành phố chưa bao giờ nói chuyện với em. </s>
[En]: i came to tokyo when i was three.
[Vi-Groundtruth]: anh đến tokyo lúc ba tuổi.
	[V

# 3. Bleu score

In [6]:
from torchtext.data.metrics import bleu_score

In [7]:
test_size = 20

preds = []
refs = []
for i in tqdm(range(test_size)):
    # Ref
    vi_utt = test_dset[i]['Y_sentence']
    refs.append( [['<s>'] + vi_utt.split(' ') + ['</s>']] )

    # Hyp
    en_utt = test_dset[i]['X_sentence']
    vi_decodes = translate(en_utt,
        model=transformer,
        en_tokenizer=en_tokenizer, vi_tokenizer=vi_tokenizer,
        Tx=Tx, Ty=Ty, beam_width=10, device=device)
    preds.append( vi_decodes[0][0].split(' ') )

bleu_score(preds, refs, max_n=4, weights=[0.25, 0.25, 0.25, 0.25])

100%|██████████| 20/20 [06:09<00:00, 18.49s/it]


0.46462185423549635